#ETL Technical Challenge

In [1]:
import pandas as pd


###Task 1: Download the data and read the CSV file

In [2]:
data_url = 'https://raw.githubusercontent.com/spunkjockey/sample-datasets/main/Datasets/Retail/OrderDetails/00.csv'
data=pd.read_csv(data_url, encoding='latin1')
data

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,DAYS_SINCE_PRIOR_ORDER,ORDER_DETAIL
0,1000867,198377,5,0,14,9.0,Triscuit Baked Whole Grain Wheat Rosemary & Ol...
1,100258,156548,29,0,23,15.0,"Stage 2 Spinach, Apple & Kale|baby food formul..."
2,1003556,4233,4,0,9,6.0,Gluten Free Rigatoni|dry pasta|13~Organic Red ...
3,1006312,7861,7,0,12,30.0,Condensed Cream of Mushroom Soup|soup broth bo...
4,1014295,144587,6,0,9,8.0,Pure Irish Butter|butter|12~Pure Cane Granulat...
...,...,...,...,...,...,...,...
27460,994536,138392,64,0,7,7.0,100% Whole Wheat Bread|bread|6~Organic Broccol...
27461,995322,151424,28,0,14,16.0,Banana|fresh fruits|7~Organic Avocado|fresh fr...
27462,995630,144400,11,0,10,7.0,Original Rotisserie Chicken|prepared meals|7
27463,996848,107747,62,0,14,3.0,Strawberries|fresh fruits|9~Organic Lactose Fr...


###Task 2: Examine the data

In [3]:
data.dtypes

ORDER_ID                    int64
USER_ID                     int64
ORDER_NUMBER                int64
ORDER_DOW                   int64
ORDER_HOUR_OF_DAY           int64
DAYS_SINCE_PRIOR_ORDER    float64
ORDER_DETAIL               object
dtype: object

In [4]:
data['DAYS_SINCE_PRIOR_ORDER'] =  data['DAYS_SINCE_PRIOR_ORDER'].astype('int64')


###Task 3: Parse the ORDER_DETAIL column

In [5]:
data['ORDER_DETAIL'][0]

'Triscuit Baked Whole Grain Wheat Rosemary & Olive Oil|crackers|8~Nutter Butter Cookies|cookies cakes|10~Chili With Beans|canned meals beans|6~Zingers Raspberry Cakes|cookies cakes|6~Ho Hos Cakes|cookies cakes|13~Small Curd Cottage Cheese|other creams cheeses|14~Original Corn Chips|chips pretzels|14~Original Citrus Sparkling Flavored Soda|soft drinks|13'

In [6]:
data['ORDER_DETAIL'].str.split('~')

0        [Triscuit Baked Whole Grain Wheat Rosemary & O...
1        [Stage 2 Spinach, Apple & Kale|baby food formu...
2        [Gluten Free Rigatoni|dry pasta|13, Organic Re...
3        [Condensed Cream of Mushroom Soup|soup broth b...
4        [Pure Irish Butter|butter|12, Pure Cane Granul...
                               ...                        
27460    [100% Whole Wheat Bread|bread|6, Organic Brocc...
27461    [Banana|fresh fruits|7, Organic Avocado|fresh ...
27462       [Original Rotisserie Chicken|prepared meals|7]
27463    [Strawberries|fresh fruits|9, Organic Lactose ...
27464    [Zero Calorie Cola|soft drinks|7, Chicken Tort...
Name: ORDER_DETAIL, Length: 27465, dtype: object

In [7]:
data['ORDER_DETAIL'] = data['ORDER_DETAIL'].str.split('~')
data = data.explode('ORDER_DETAIL')
data

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,DAYS_SINCE_PRIOR_ORDER,ORDER_DETAIL
0,1000867,198377,5,0,14,9,Triscuit Baked Whole Grain Wheat Rosemary & Ol...
0,1000867,198377,5,0,14,9,Nutter Butter Cookies|cookies cakes|10
0,1000867,198377,5,0,14,9,Chili With Beans|canned meals beans|6
0,1000867,198377,5,0,14,9,Zingers Raspberry Cakes|cookies cakes|6
0,1000867,198377,5,0,14,9,Ho Hos Cakes|cookies cakes|13
...,...,...,...,...,...,...,...
27463,996848,107747,62,0,14,3,Ice Cream Tahitian Vanilla|ice cream ice|10
27463,996848,107747,62,0,14,3,Organic Cheddar Bunnies|crackers|13
27463,996848,107747,62,0,14,3,Sparkling Water Grapefruit|water seltzer spark...
27464,999604,195396,4,0,9,30,Zero Calorie Cola|soft drinks|7


In [8]:

data[['PROD','AISLE','SEQ_PROD']] =data['ORDER_DETAIL'].str.split('|',expand=True)
data

,ORDER_ID,USER_ID,ORDER_NUMBER,ORDER_DOW,ORDER_HOUR_OF_DAY,DAYS_SINCE_PRIOR_ORDER,ORDER_DETAIL,PROD,AISLE,SEQ_PROD
0,1000867,198377,5,0,14,9,Triscuit Baked Whole Grain Wheat Rosemary & Ol...,Triscuit Baked Whole Grain Wheat Rosemary & Ol...,crackers,8
0,1000867,198377,5,0,14,9,Nutter Butter Cookies|cookies cakes|10,Nutter Butter Cookies,cookies cakes,10
0,1000867,198377,5,0,14,9,Chili With Beans|canned meals beans|6,Chili With Beans,canned meals beans,6
0,1000867,198377,5,0,14,9,Zingers Raspberry Cakes|cookies cakes|6,Zingers Raspberry Cakes,cookies cakes,6
0,1000867,198377,5,0,14,9,Ho Hos Cakes|cookies cakes|13,Ho Hos Cakes,cookies cakes,13
...,...,...,...,...,...,...,...,...,...,...
27463,996848,107747,62,0,14,3,Ice Cream Tahitian Vanilla|ice cream ice|10,Ice Cream Tahitian Vanilla,ice cream ice,10
27463,996848,107747,62,0,14,3,Organic Cheddar Bunnies|crackers|13,Organic Cheddar Bunnies,crackers,13
27463,996848,107747,62,0,14,3,Sparkling Water Grapefruit|water seltzer spark...,Sparkling Water Grapefruit,water seltzer sparkling water,7
27464,999604,195396,4,0,9,30,Zero Calorie Cola|soft drinks|7,Zero Calorie Cola,soft drinks,7


In [9]:
data = data.drop(columns=['ORDER_DETAIL'])

### Task 4: Load the data to BigQuery

In [10]:

from google.colab import auth
auth.authenticate_user()

In [11]:
from google.cloud import bigquery
project_id = 'tokyo-data-313702'
client = bigquery.Client(project=project_id)

In [12]:
table_id = 'applaudo.orders'
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("ORDER_ID", "INT64"),
    bigquery.SchemaField("USER_ID", "INT64"),
    bigquery.SchemaField("ORDER_NUMBER", "INT64"),
    bigquery.SchemaField("ORDER_DOW", "INT64"),
    bigquery.SchemaField("ORDER_HOUR_OF_DAY", "INT64"),
    bigquery.SchemaField("DAYS_SINCE_PRIOR_ORDER", "FLOAT64"),
    bigquery.SchemaField("PROD", "STRING"),
    bigquery.SchemaField("AISLE", "STRING"),
    bigquery.SchemaField("SEQ_PROD", "STRING"),
])
job = client.load_table_from_dataframe(
    data, table_id, job_config=job_config
)
job.result()

###Task 5: Perform a count of rows

In [13]:
sql = """
    SELECT COUNT(*)
    FROM `{}.{}`
    
    """.format(project_id,table_id)
result = pd.read_gbq(sql, project_id=project_id)
print("Check: {}".format(result))

Check:       f0_
0  324026


### Task 6: Analize

The mix of aisles with more products vs products sold

In [14]:
sql = """
WITH aisle_more_sold AS(
SELECT 
AISLE,
COUNT(ORDER_ID) Orders_Sold
FROM  `tokyo-data-313702.applaudo.orders` 
GROUP BY  AISLE
ORDER BY COUNT(ORDER_ID)  DESC
LIMIT 10
),
aisle_more_products AS(
SELECT 
AISLE,
COUNT(DISTINCT(PROD)) Products
FROM  `tokyo-data-313702.applaudo.orders` 
GROUP BY  AISLE
ORDER BY count(DISTINCT(PROD)) DESC
LIMIT 10
)

SELECT 
aisle_more_products.AISLE,
Products,
Orders_Sold 
FROM aisle_more_products 
JOIN aisle_more_sold 
ON aisle_more_products.AISLE = aisle_more_sold.AISLE
"""
result = pd.read_gbq(sql, project_id=project_id)
print(result)

                        AISLE  Products  Orders_Sold
0                      yogurt       710        13398
1              chips pretzels       601         6521
2             packaged cheese       599         9953
3  packaged vegetables fruits       455        20244
4            fresh vegetables       442        39998


The days of the week with the highest number of orders processed

In [15]:
sql = """
SELECT 
CASE 
    WHEN ORDER_DOW = 0 THEN 'Sunday'
    WHEN ORDER_DOW = 1 THEN 'Monday'
    WHEN ORDER_DOW = 2 THEN 'Tuesday'
    WHEN ORDER_DOW = 3 THEN 'Wednesday'
    WHEN ORDER_DOW = 4 THEN 'Thursday'
    WHEN ORDER_DOW = 5 THEN 'Friday'
    WHEN ORDER_DOW = 6 THEN 'Saturday'
  END Weekday,
COUNT(DISTINCT(ORDER_ID)) AS Orders
FROM  `tokyo-data-313702.applaudo.orders` 
GROUP BY ORDER_DOW 
ORDER BY COUNT(DISTINCT(ORDER_ID)) DESC
LIMIT 1;"""
result = pd.read_gbq(sql, project_id=project_id)
print(result)

  Weekday  Orders
0  Sunday   27465


View the number of orders per day and hour

In [16]:
sql = """
SELECT 
CASE 
    WHEN ORDER_DOW = 0 THEN 'Sunday'
    WHEN ORDER_DOW = 1 THEN 'Monday'
    WHEN ORDER_DOW = 2 THEN 'Tuesday'
    WHEN ORDER_DOW = 3 THEN 'Wednesday'
    WHEN ORDER_DOW = 4 THEN 'Thursday'
    WHEN ORDER_DOW = 5 THEN 'Friday'
    WHEN ORDER_DOW = 6 THEN 'Saturday'
  END Weekday,
  ORDER_HOUR_OF_DAY Hour,
COUNT(DISTINCT(ORDER_ID)) AS Orders
FROM  `tokyo-data-313702.applaudo.orders` 
GROUP BY ORDER_DOW , ORDER_HOUR_OF_DAY

"""
result = pd.read_gbq(sql, project_id=project_id)
print(result)

   Weekday  Hour  Orders
0   Sunday     0     148
1   Sunday     1     101
2   Sunday     2      77
3   Sunday     3      45
4   Sunday     4      32
5   Sunday     5      66
6   Sunday     6     116
7   Sunday     7     567
8   Sunday     8    1222
9   Sunday     9    1774
10  Sunday    10    2245
11  Sunday    11    2354
12  Sunday    12    2415
13  Sunday    13    2417
14  Sunday    14    2593
15  Sunday    15    2518
16  Sunday    16    2243
17  Sunday    17    1800
18  Sunday    18    1408
19  Sunday    19    1042
20  Sunday    20     811
21  Sunday    21     650
22  Sunday    22     505
23  Sunday    23     315
24  Sunday    24       1
